In [162]:
import undetected_chromedriver as uc
from selenium.webdriver.common.by import By
from selenium.webdriver.support.wait import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
import cv2
import numpy as np
import matplotlib.pyplot as plt
import ddddocr


ModuleNotFoundError: No module named 'ddddocr'

In [96]:
options = uc.ChromeOptions()
url = 'https://tixcraft.com/activity/detail/23_abc30'
driver = uc.Chrome()
driver.maximize_window()
driver.get(url)

In [102]:
def wait_until(css_selector, wait=10, mode=0, all=False):
    if(mode == 0 and not all):
        return WebDriverWait(driver, wait).until(EC.presence_of_element_located((By.CSS_SELECTOR, css_selector)))
    elif(mode == 0 and all):
        return WebDriverWait(driver, wait).until(EC.presence_of_all_elements_located((By.CSS_SELECTOR, css_selector)))
    elif(mode == 1):
        return WebDriverWait(driver, wait).until(EC.visibility_of_element_located((By.CSS_SELECTOR, css_selector)))
    elif(mode == 0 and all):
        return WebDriverWait(driver, wait).until(EC.visibility_of_all_elements_located((By.CSS_SELECTOR, css_selector)))


In [106]:
creds = []
with open('./credentials') as f:
    for cred in f.readlines():
        creds.append(cred.strip().split('=')[-1])
email, pw = creds

In [98]:
wait_until("button[id='onetrust-reject-all-handler']", mode=1).click()
wait_until("a[href='#login']").click()
wait_until("a[id='loginGoogle']", mode=1).click()
wait_until("input[type='email']", mode=1).send_keys(email)
wait_until("button[jsname='LgbsSe'][data-idom-class='nCP5yc AjY5Oe DuMIQc LQeN7 qIypjc TrZEUc lw1w4b']").click()
wait_until("input[type='password']", mode=1).send_keys(pw)
wait_until("button[jsname='LgbsSe'][data-idom-class='nCP5yc AjY5Oe DuMIQc LQeN7 qIypjc TrZEUc lw1w4b']").click()


In [134]:
wait_until("a[target='_new']").click()

In [126]:
event_name = 'Palestine - CHINESE TAIPEI'
event_date_time = ['2023/12/05', '18:30']

In [135]:
found_match_event = False
for i, tr in enumerate(wait_until('tbody > tr', all=True)):
    dt_check, event_check = False, False
    for j, td in enumerate(tr.find_elements(By.CSS_SELECTOR, 'td')):
        if(j == 0):
            row_texts = td.text.split(' ')
            row_texts.pop(1)
            if(row_texts != event_date_time):
                break
            else:
                dt_check = True
        elif(j == 1):
            if(td.text != event_name):
                break
            else:
                event_check = True
        if(dt_check and event_check):
            event = tr
            found_match_event = True
    if(found_match_event):
        break
event.find_elements(By.CSS_SELECTOR, 'td')[-1].click()
        

In [156]:
section_name = '南F區'
price = '500'
ticket_amount = 4

In [155]:
found_match_section = False
for group in wait_until("div[class='zone area-list'] > ul", all=True):
    for section in group.find_elements(By.CSS_SELECTOR, "li"):
        if(section.text.find(section_name) != -1):
            chosen_section = section.find_element(By.CSS_SELECTOR, "a")
            found_match_section = True
            break
    if(found_match_section):
        break
chosen_section.click()
            


In [158]:
wait_until(f"option[value='{ticket_amount}']", all=True)[0].click()

In [161]:
n = 3
with open('captcha.png', 'wb') as f:
    f.write(wait_until("img[id='TicketForm_verifyCode-image']").screenshot_as_png)
img = cv2.imread('captcha.png', 0)
img = cv2.erode(img, np.ones((n, n), np.uint8)) 
img = cv2.bitwise_not(img)
cv2.imwrite('captcha_g.png', img)
ocr = ddddocr.DdddOcr(show_ad=False)
with open('captcha_g.png', 'rb') as f:
    image_bytes = f.read()
res = ocr.classification(image_bytes)
print(res)